## UK Deaths by Vaccination Status


In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

data = pd.read_csv('table.csv')
df = pd.DataFrame(data, columns=['Year', 'Month', 'Vaccination status', 'Age-standardised mortality rate / 100,000 person-years'])

# pivot table to break status out as columns
columns = ['Date']
for i, row in df.iterrows():
    status = row['Vaccination status']
    if not status in columns:
        columns += [status]   

dp = pd.DataFrame(columns=columns)

data = {}
for i, row in df.iterrows():
    year = row['Year']
    month = row['Month']
    status = row['Vaccination status']
    rate = row['Age-standardised mortality rate / 100,000 person-years']
    if rate == 'x':
        rate = 0
    rate = float(rate)
    
    if status == 'Unvaccinated':
        data['Date'] = f"{year}.{month}"

    data[status] = rate

    if status == 'Ever vaccinated':
        dp = dp.append(data, ignore_index=True)
        data = {}

dates = dp[columns[0]]
xs = list(range(len(dates)))

averages = np.zeros(len(dates))
count = 0
for i in range(2, len(columns)):
    unvax = dp[columns[1]]
    label = columns[i]
    vax = dp[label]
    vs = np.array(vax)
    if label != 'Ever vaccinated':
        averages += vs
        count += 1
    p = figure(width=1000, height=400) 
    p.line(xs, unvax, color='blue', legend_label=columns[1])
    p.line(xs, vax, color='green', legend_label=label)
    show(p)

averages /= count # average back to per 100k
p = figure(width=1000, height=400) 
p.line(xs, unvax, color='blue', legend_label=columns[1])
p.line(xs, averages, color='green', legend_label="Average vaccinated")
show(p)


Loading BokehJS ...